# About the experiment
Aim: Determine if we can differentiate between 0, 1, 2 CNOTs being applied at the same time. Bonus if we can determine which CNOT is being applied exactly.
Try both phase flips and amplitude flips attack model.

To find out we take a 4 qubit chain for the victim qubits. That means there are 3 CNOT links and each link can be (OFF, +, -), where + and - signify the direction of the CNOT.

1. No CNOTs - 1
2. 1 CNOT - ${{3}\choose{1}} \times 2 = 6$
3. 2 CNOTs - 4\
    We might think ${{3}\choose{2}} \times 2 = 6$ \
    But, we cannot have common qubits. So, we can only select the first and third link. Therefore, we get 4 possibilities.



Backend: IBM Lagos.

On this backend we can choose 4 such 4 qubit chains.
Since we want to be able to detect upto two CNOTs we shall snoop for about 2 CNOT durations.

# Imports

In [ ]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from qiskit.transpiler import CouplingMap
from qiskit.circuit import Parameter

import common.attacker as attack_helper

# Create Victim Circuits

In [ ]:
victim_qregs = QuantumRegister(4, name='vic')

In [ ]:
victim_cm = CouplingMap([
    [0, 1],
    [1, 2],
    [2, 3]
])

victim_cm.make_symmetric()

In [ ]:
display(victim_cm.draw().convert('RGB'))

In [ ]:
list(victim_cm.get_edges())

## No CNOTs

In [ ]:
def no_cnot_vic():
    qc = QuantumCircuit(victim_qregs, name='no_cnots_victim')
    return qc

## 1 CNOT

In [ ]:
def one_cnot_vic():
    qcs = []
    for cx_edge in victim_cm.get_edges():
        qc = QuantumCircuit(victim_qregs, name='one_cnot_victim_{}'.format(cx_edge))
        qc.cx(*cx_edge)
        # qc.cx(cx_edge[0], cx_edge[1])
        qcs.append(qc)
    return qcs

## 2 CNOTs

In [ ]:
def two_cnot_vic():
    group_1 = [(0, 1), (1, 0)] 
    group_2 = [(2, 3), (3, 2)]
    qcs = []
    for i in group_1:
        for j in group_2:
            qc = QuantumCircuit(victim_qregs)
            qc.cx(*i)
            qc.cx(*j)
            qcs.append(qc)
    return qcs

## Consolidate

In [ ]:
no_cnot_vic_qc = no_cnot_vic()
one_cnot_vic_qcs = one_cnot_vic()
two_cnot_vic_qcs = two_cnot_vic()

# Create Attack Circuits

In [ ]:
reps = 2
attack_qregs = QuantumRegister(5)
attack_cregs = ClassicalRegister(5 * reps)
snooping_duration_param = Parameter("snooping_duration")

In [ ]:
attack_phase_flips_qc = attack_helper.attack_circ_phase_flips(attack_qregs, attack_cregs, snooping_duration_param, reps)

In [ ]:
attack_phase_flips_qc.draw('mpl')

In [ ]:
# def attack_circ_amplitude_flips(attack_qregs, attack_cregs, snooping_duration_param):
#     attack_qc_z_flips = QuantumCircuit(attack_qregs, attack_cregs, name="Attack Circ")
#     attack_qc_z_flips.delay(snooping_duration_param)
#     attack_qc_z_flips.measure(attack_qregs, attack_cregs)
#     return attack_qc_z_flips

In [ ]:
# def attack_circ_phase_flips(attack_qregs, attack_cregs, snooping_duration_param):
#     attack_qc_x_flips = QuantumCircuit(attack_qregs, attack_cregs)
#     attack_qc_x_flips.h(attack_qregs)
#     attack_qc_x_flips.delay(snooping_duration_param)
#     attack_qc_x_flips.h(attack_qregs)
#     attack_qc_x_flips.measure(attack_qregs, attack_cregs)
#     return attack_qc_x_flips